# Doctor-Hospital Matching With Hungarian Algorithm

#### N doctors are matched to K hospitals with this algorithm. The inputs given are:
             1)Number of doctors
             2)Number of hospitals
             3)Capacity of hospitals
             4)Preference of each doctor
             
#### The assignment of doctors to hospitals is the result.

Import numpy and pandas modules

In [1]:
import numpy as np  
import pandas as pd

Input the number of doctors, hospitals and capacity of the hospital

In [2]:
doc = int(input("Enter number of doctors:"))
hosp = int(input("Enter number of hospitals:"))
cap = int(input("Enter the capacity of the hospitals:"))

Enter number of doctors:4
Enter number of hospitals:3
Enter the capacity of the hospitals:2


Input the preference of each doctor

In [3]:
preference = []
p = []
for i in range(doc):
    p = eval(input("Enter preference for doctor " + str(i+1) + ":"))
    preference.append(p)
print(preference)

Enter preference for doctor 1:1,2,3
Enter preference for doctor 2:3,2,1
Enter preference for doctor 3:2,1,3
Enter preference for doctor 4:3,1,2
[(1, 2, 3), (3, 2, 1), (2, 1, 3), (3, 1, 2)]


Create a matrix with doctors' preferences.

In [4]:
matrix_1 = np.array(preference)
matrix_2 = np.tile(matrix_1,cap)
print(matrix_2)

[[1 2 3 1 2 3]
 [3 2 1 3 2 1]
 [2 1 3 2 1 3]
 [3 1 2 3 1 2]]


If the matrix is rectangular, modify it into a n x n square matrix and assign a number higher than the number of hospitals to the dummy row elements.

In [5]:
row_num,col_num = np.shape(matrix_2)
if row_num != col_num:
    d = hosp+1
    if row_num < col_num:
        arr = np.array([d for _ in range(col_num)])
        x=np.stack([arr for _ in range(col_num-row_num)], axis=0)
        matrix_2 = np.append(matrix_2,x,axis=0)
    else:
        arr = np.array([d for _ in range(row_num)])
        x=np.stack([arr for _ in range(row_num-col_num)], axis=1)
        matrix_2 = np.append(matrix_2,x,axis=1)  
print(matrix_2)

[[1 2 3 1 2 3]
 [3 2 1 3 2 1]
 [2 1 3 2 1 3]
 [3 1 2 3 1 2]
 [4 4 4 4 4 4]
 [4 4 4 4 4 4]]


Now the hungarian algorithm is implemented. 

Subtract row minima and row maxima

In [6]:
row_num,col_num = np.shape(matrix_2)
for r in range(row_num):
    matrix_2[r,:]=matrix_2[r,:]-np.min(matrix_2[r,:])

for cl in range(col_num):
    matrix_2[:,cl]=matrix_2[:,cl]-np.min(matrix_2[:,cl])
    
print(matrix_2)

[[0 1 2 0 1 2]
 [2 1 0 2 1 0]
 [1 0 2 1 0 2]
 [2 0 1 2 0 1]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


Cover all zeros with a minimum number of lines, create additional zeros if the number of lines is less than n. Optimal assignment is then implemented.

In [7]:
zero_index = np.argwhere(matrix_2 == 0)
print(zero_index)

[[0 0]
 [0 3]
 [1 2]
 [1 5]
 [2 1]
 [2 4]
 [3 1]
 [3 4]
 [4 0]
 [4 1]
 [4 2]
 [4 3]
 [4 4]
 [4 5]
 [5 0]
 [5 1]
 [5 2]
 [5 3]
 [5 4]
 [5 5]]


In [8]:
list_x =[]
list_y = []

for i in range(len(zero_index)):

    for j in range(i + 1 ,len(zero_index)):
        if zero_index[i,0] == zero_index[j,0]:
            
            list_x.append(zero_index[i,0])
            n =  1
        elif zero_index[i,1] == zero_index[j,1]:
            n = 1
            list_y.append(zero_index[i,1])

In [9]:
#determine the end
def end_loop(matrix_2):

    row_uncovered = np.ones(row_num, dtype=bool)
    col_uncovered = np.ones(col_num, dtype=bool)

    marked = np.zeros((row_num, col_num), dtype=int)

    for i, j in zip(*np.where(matrix_2 == 0)):
        if col_uncovered[j] and row_uncovered[i]:
            marked[i, j] = 1
            col_uncovered[j] = False
            row_uncovered[i] = False

    row_uncovered[:] = True
    col_uncovered[:] = True

    marked = (marked == 1)
    col_uncovered[np.any(marked, axis=0)] = False

    if marked.sum() < row_num:
        return 1
    else:
        return 0
    
flag = end_loop(matrix_2)
print(flag)

0


In [10]:
#optimal assignment

def same_row_col(zero_index):
    list_m =[]
    list_n = []


    for i in range(len(zero_index)):
        for j in range(i+1 ,len(zero_index)):
            if zero_index[i,0] == zero_index[j,0]:          
                list_m.append(zero_index[i,0])
            elif zero_index[i,1] == zero_index[j,1]:
                list_n.append(zero_index[i,1])

    
           

    list_a = []
    list_b = []
    for i in range(len(zero_index)):
        if not(zero_index[i,0] in list_m and zero_index[i,1] in list_n):
            list_a.append(zero_index[i,0])
            list_b.append(zero_index[i,1])
 
    for i in range(len(zero_index)):
        for j in range(i+1 ,len(zero_index)):
            if zero_index[i,0] in list_m and zero_index[i,1] in list_n:          
                list_a.append(zero_index[i,0])
                list_b.append(zero_index[i,1])
                list_m.remove(zero_index[i,0])
                list_n.remove(zero_index[i,1])
 
                

    return list_a, list_b

                
        

def final_assignment(matrix_3):
    zero_index = np.argwhere(matrix_3 == 0)
    
    list_doctor = []
    list_hospital = []


    list_doctor, list_hospital = same_row_col(zero_index)

 
    return list_doctor, list_hospital

In [11]:
if flag:

    while (flag):

        #find the single zero index, not in the same row or same col with other zero
        list_single_x = []
        list_single_y = []
        for i in range(len(zero_index)):
            if zero_index[i,0] not in list_x and zero_index[i,1] not in list_y:
                list_single_x.append(zero_index[i,0])
                list_single_y.append(zero_index[i,1])
                

        
        #find the min uncovered number and its index
        min_update = matrix_2.max()
        min_row_index = 0
        min_col_index = 0
        for i in range(row_num):
            for j in range(col_num):
                if i not in list_x and j not in list_y:
                    if matrix_2[i,j]!=0 and matrix_2[i,j] < min_update:
                        min_update = matrix_2[i,j]
                        min_row_index = i
                        min_col_index = j
            
   
        # determine row or col of single zero which to use, and figure out the covered rows and cols
        for i in range(len(list_single_x)):
            if list_single_x[i] == min_row_index:
                list_y.append(list_single_y[i])
            else:
                list_x.append(list_single_x[i])
      
        
        #substract the min num from all uncovered elements and add it to all elements that are covered twice. Then get the updated matrix.
       
        for i in range(row_num):
            for j in range(col_num):
                if i in list_x and j in list_y:
                    matrix_2[i,j] = matrix_2[i,j] + min_update
                elif i not in list_x and j not in list_y:
                    matrix_2[i,j] = matrix_2[i,j] - min_update
                    
        flag = end_loop(matrix_2)

The final lists of assignment are returned.

In [12]:
doc_final, hosp_final = final_assignment(matrix_2)

The dummy assignments are discarded and the final assignment is displayed.

In [13]:
doc_final = [x+1 for x in doc_final]
hosp_final = [x+1 for x in hosp_final]
doc_final = doc_final[:doc]
hosp_final = hosp_final[:doc]
print(doc_final, hosp_final)
df = pd.DataFrame({'Doctor' : doc_final[:doc], 'Hospital' : hosp_final[:doc]})   
print(df)

[1, 2, 3, 4] [1, 3, 2, 2]
   Doctor  Hospital
0       1         1
1       2         3
2       3         2
3       4         2
